In [14]:
%load_ext autotime

time: 128 µs (started: 2023-03-23 20:25:43 +05:30)


In [17]:
def add(ciX, ciY):
# fixed iteration since
    
    for i in range(size):
        a = vm.gate_and(ciX, ciY)
        b = vm.gate_xor(ciX, ciY)
        aShiftTemp = a
        # using roll as a shift bit
        aShiftTemp.roll(1, axis=0)
        #aShiftTemp=np.roll(aShiftTemp,-1, axis=0)
        ciX = aShiftTemp
        ciY = b
    #result=ctx.decrypt(secret_key,b)
    #print("result is",boolListToInt(result))
    return b
def add_mul(ciX, ciY):
# fixed iteration since
    
    for i in range(size-1):
        a = vm.gate_and(ciX, ciY)
        b = vm.gate_xor(ciX, ciY)
        aShiftTemp = a
        # using roll as a shift bit
        aShiftTemp.roll(1, axis=0)
        #aShiftTemp=np.roll(aShiftTemp,-1, axis=0)
        ciX = aShiftTemp
        ciY = b
    #result=ctx.decrypt(secret_key,b)
    #val=listToString(ctx.decrypt(secret_key, b))
    #print("cix in addmul:",two2dec(val[::-1]))
    #print("result is",boolListToInt(result))
    return b
def boolListToInt(bitlists):
    out = 0
    bitlists[:] = bitlists[::-1]
    for bit in bitlists:
        out = (out << 1) | bit
    return out
'''def boolListToInt(bitlists):
    out = 0
    print(type(bitlists))
    np.roll(bitlists,7,axis=0)
    for bit in bitlists:
        out = (out << 1) | bit
    return out'''
def fixSizeBoolList(decimal,size):
    x = [[int(x)] for x in bin(decimal)[2:]]
    for i in range(len(x)):
        x[i] = list(map(bool, x[i]))
    x = [[False]]*(size - len(x)) + x
    x.reverse()
    return x

def mul(ciX,ciY,enc_temp):
    
    msb_temp= [vm.empty_ciphertext((1,))]
    control=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    c_s=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    c_s1=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    msb_temp=vm.gate_xor(ciX[size-1],ciY[size-1])
    t=size-1
    for i in range(size):
            control[i]=ciX[t]
            c_s1[i]=ciY[t]
    mul_temp=ciX
    zero=enc_temp
    one_ciX=vm.gate_not(ciX)
    result=ctx.decrypt(secret_key, one_ciX)
    one_ciY=vm.gate_not(ciY)
    comp_ciX=add(one_ciX,one)
    comp_ciY=add(one_ciY,one)
    
    #print("control bit:",ctx.decrypt(secret_key, control))
    ciX=mux(control,comp_ciX,ciX)
    ciY=mux(c_s1,comp_ciY,ciY)
    #print("cix:",ctx.decrypt(secret_key, ciX))
    for j in range(0,size):
        #print("=================iteration==========",j)
        
        
        #print("cix:",ctx.decrypt(secret_key, ciX))
        for i in range(size):
            c_s[i]=ciY[j]
        add_res=mux(c_s,ciX,zero)
        
        enc_theta=add(enc_temp,add_res)
        enc_temp=enc_theta
        #enc_temp=ciX
        #print("enc_teta:",ctx.decrypt(secret_key, enc_theta))
        
        #print("control bit:",ctx.decrypt(secret_key, c_s[0]))
        #mux_result=mux(c_s,enc_theta,ciX)
        #mux_result=vm.gate_mux(c_s, enc_theta,ciX)
        #print("mux result:",ctx.decrypt(secret_key, mux_result))
        #ciX=mux_result
        ciX.roll(1, axis=0)
        #ciX.roll(1, axis=0)
    for i in range(size):
        c_s[i]=msb_temp    
    one_presult=vm.gate_not(enc_theta)
    comp_presult=add(one_presult,one)
    ciX=vm.gate_mux(c_s,comp_presult,enc_theta)
    return ciX




def Convert_list(string):
    list1=[]
    list1[:0]=string
    #print(list1)
    list1=[int(i)for i in list1 ]
    listb=[]
    for i in list1:
        if i==0:
            listb.append([False])
        else:
            listb.append([True])    
    
    #print(listb)
    return listb
def twos_complement(n,nbits):
    a=f"{n & ((1 << nbits) - 1):0{nbits}b}" 
    #print(type(a))
    a=Convert_list(a)
    a.reverse()
    return a
def listToString(s): 
    # initialize an empty string
    list1=[int(i)for i in s ]
    listp=[]
    for i in list1:
        if i==False:
            listp.append('0')
        else:
            listp.append('1')    
    
    #print(listp)
    str1 = "" 
    # traverse in the string  
    s=['delim'.join([str(elem) for elem in sublist]) for sublist in listp]
    #print(s)
    for ele in s: 
        str1 += ele  
    # return string 
    #print(str1)
    return str1
def mux(c_s,true,false) :
    temp1=vm.gate_and(c_s,true)
    #print("temp1:",ctx.decrypt(secret_key, temp1))
    temp2=vm.gate_and(vm.gate_not(c_s),false)
    #print("temp2:",ctx.decrypt(secret_key, temp2))
    mux_result=vm.gate_or(temp1,temp2)
    return mux_result
def two2dec(s):
    if s[0] == '1':
        return -1 * (int(''.join('1' if x == '0' else '0' for x in s), 2) + 1)
    else:
        return int(s, 2)

time: 2.85 ms (started: 2023-03-23 20:28:22 +05:30)


In [24]:
import random
import nufhe
import time
import numpy as np
ctx = nufhe.Context()
secret_key, cloud_key = ctx.make_key_pair()
vm = ctx.make_virtual_machine(cloud_key)
size = 16
n=twos_complement(14,size)
print("n is ",n)
p=twos_complement(1,size)
print("p is ",p)
ciX=ctx.encrypt(secret_key, n)
ciY=ctx.encrypt(secret_key, p)
one=ctx.encrypt(secret_key, twos_complement(1,size))
x=twos_complement(-128,size)
#print("msb to add",x)
oneroll=ctx.encrypt(secret_key, x)

temp=fixSizeBoolList(0,size)
enc_temp=ctx.encrypt(secret_key, temp)

s=[ [False] for i in range(size)]
c_s=ctx.encrypt(secret_key, s)
'''c_s1=ctx.encrypt(secret_key, s)
result=ctx.decrypt(secret_key,ciX)
for i in range(size):
    c_s[i]=ciX[size-1]
    c_s1[i]=ciY[size-1]
    
    
msb_temp= [vm.empty_ciphertext((1,))]
msb_temp=vm.gate_xor(ciX[size-1],ciY[size-1])    
    
print("cix is",boolListToInt(result))
one_ciX=vm.gate_not(ciX)
result=ctx.decrypt(secret_key, one_ciX)
print("ones compelent",boolListToInt(result))
#one_ciX=vm.gate_xor(one_ciX,oneroll)
#one_ciX[size-1]=vm.gate_xor(one_ciX[size-1],one[0])
one_ciY=vm.gate_not(ciY)
#one_ciY=vm.gate_xor(one_ciY,oneroll)
#one_ciY[size-1]=vm.gate_xor(one_ciY[size-1],one[0])
comp_ciX=add(one_ciX,one)

comp_ciY=add(one_ciY,one)

    
ciX=mux(c_s,comp_ciX,ciX)
ciY=mux(c_s1,comp_ciY,ciY)



result=ctx.decrypt(secret_key,ciX)
print("cix before multiplication",boolListToInt(result))
result=ctx.decrypt(secret_key,ciY)
print("ciy before multiplication",boolListToInt(result))


presult_mul1 = mul(ciX,ciY,enc_temp) #multiplication


for i in range(size):
    c_s[i]=msb_temp
result=ctx.decrypt(secret_key,c_s)
print("control bit",result)    
one_presult=vm.gate_not(presult_mul1)
comp_presult=add(one_presult,one)
result=ctx.decrypt(secret_key,comp_presult)
print("twos complemnt",boolListToInt(result))
ciX=vm.gate_mux(c_s,comp_presult,presult_mul1)'''

ciX=mul(ciX,ciY,enc_temp)
#ciX=add(ciX,ciY)
result=ctx.decrypt(secret_key, ciX)
print(result)
#np.roll(result,size,axis=0)

#print("mux result:",)

val=listToString(result)
print(val)
print("2scomplement to dec ",two2dec(val[::-1]) )   
print("twos complemnt",boolListToInt(result))

n is  [[False], [True], [True], [True], [False], [False], [False], [False], [False], [False], [False], [False], [False], [False], [False], [False]]
p is  [[True], [False], [False], [False], [False], [False], [False], [False], [False], [False], [False], [False], [False], [False], [False], [False]]
[[False]
 [ True]
 [ True]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]]
0111000000000000
2scomplement to dec  14
twos complemnt [14]
time: 1min 15s (started: 2023-03-23 23:27:58 +05:30)
